In [1]:
from abc import ABCMeta, abstractmethod
from collections import namedtuple
from itertools import chain, repeat
from pprint import pformat, pprint
from typing import (
    AbstractSet, Any, Callable, Collection, Dict, Generator, Generic, Iterable, List, Mapping,
    NamedTuple, Optional, Tuple, TypeVar, Union
)

from graphql import GraphQLList, GraphQLString, parse
from graphql.utilities.build_ast_schema import build_ast_schema
from graphql_compiler.compiler.blocks import (
    Backtrack, CoerceType, ConstructResult, EndOptional, Filter, GlobalOperationsStart, 
    MarkLocation, Recurse, QueryRoot, Traverse
)
from graphql_compiler.compiler.compiler_entities import BasicBlock, Expression
from graphql_compiler.compiler.expressions import (
    BinaryComposition, ContextField, ContextFieldExistence, Literal, LocalField, 
    OutputContextField, TernaryConditional, Variable
)
from graphql_compiler.compiler.compiler_frontend import IrAndMetadata, graphql_to_ir
from graphql_compiler.compiler.helpers import Location, get_only_element_from_collection
from graphql_compiler.interpreter.api import interpret_ir
from graphql_compiler.interpreter.debugging import InterpreterAdapterTap, TraceReplayAdapter
from graphql_compiler.interpreter.typedefs import DataContext, EdgeInfo, InterpreterAdapter
from graphql_compiler.schema import GraphQLDate, GraphQLDateTime, GraphQLDecimal
from graphql_compiler.tests.test_helpers import SCHEMA_TEXT

In [2]:
vertices = {
    'Animal': [
        {'name': 'Scooby Doo', 'uuid': '1001'},
        {'name': 'Hedwig', 'uuid': '1002'},
        {'name': 'Beethoven', 'uuid': '1003'},
        {'name': 'Pongo', 'uuid': '1004'},
        {'name': 'Perdy', 'uuid': '1005'},
        {'name': 'Dipstick', 'uuid': '1006'},
        {'name': 'Dottie', 'uuid': '1007'},
        {'name': 'Domino', 'uuid': '1008'},
        {'name': 'Little Dipper', 'uuid': '1009'},
        {'name': 'Oddball', 'uuid': '1010'},
    ],
}
edges = {
    'Animal_ParentOf': [
        ('1004', '1006'),
        ('1005', '1006'),
        ('1006', '1008'),
        ('1006', '1009'),
        ('1006', '1010'),
        ('1007', '1008'),
        ('1007', '1009'),
        ('1007', '1010'),
    ],
}

vertices_by_uuid = {
    vertex['uuid']: vertex
    for vertex in chain.from_iterable(vertices.values())
}


class InMemoryAdapter(InterpreterAdapter[dict]):
    def get_tokens_of_type(
        self,
        type_name: str, 
        **hints
    ) -> Iterable[dict]:
        return vertices[type_name]

    def project_property(
        self,
        data_contexts: Iterable[DataContext],
        current_type_name: str,
        field_name: str,
        **hints
    ) -> Iterable[Tuple[DataContext, Any]]:
        for data_context in data_contexts:
            current_token = data_context.current_token
            current_value = current_token[field_name] if current_token is not None else None
            yield (data_context, current_value)

    def project_neighbors(
        self,
        data_contexts: Iterable[DataContext], 
        current_type_name: str,
        edge_info: EdgeInfo,
        **hints
    ) -> Iterable[Tuple[DataContext, Iterable[dict]]]:
        direction, edge_name = edge_info
        edge_data = edges[edge_name]
        
        for data_context in data_contexts:
            neighbor_tokens = []
            current_token = data_context.current_token
            if current_token is not None:
                uuid = current_token['uuid']
                if direction == 'out':
                    neighbor_tokens = [
                        vertices_by_uuid[destination_uuid]
                        for source_uuid, destination_uuid in edge_data
                        if source_uuid == uuid
                    ]
                elif direction == 'in':
                    neighbor_tokens = [
                        vertices_by_uuid[source_uuid]
                        for source_uuid, destination_uuid in edge_data
                        if destination_uuid == uuid
                    ]
                else:
                    raise AssertionError()
                
            yield (data_context, neighbor_tokens)

    def can_coerce_to_type(
        self,
        data_contexts: Iterable[DataContext], 
        current_type_name: str,
        coerce_to_type_name: str,
        **hints
    ) -> Iterable[Tuple[DataContext, bool]]:
        # TODO(predrag): See if a redesign can make this be a no-op again.
        return zip(data_contexts, repeat(True))

In [3]:
schema = build_ast_schema(parse(SCHEMA_TEXT))

In [4]:
def recursion_sanity_check():
    depth_2 = """
{
    Animal {
        name @output(out_name: "start")
        
        out_Animal_ParentOf @recurse(depth: 2) {
            name @output(out_name: "first_recursion")
            
            in_Animal_ParentOf @recurse(depth: 2) {
                name @output(out_name: "second_recursion")
            }
        }
    }
}"""
    depth_3 = """
{
    Animal {
        name @output(out_name: "start")
        
        out_Animal_ParentOf @recurse(depth: 3) {
            name @output(out_name: "first_recursion")
            
            in_Animal_ParentOf @recurse(depth: 3) {
                name @output(out_name: "second_recursion")
            }
        }
    }
}"""
    depth_4 = """
{
    Animal {
        name @output(out_name: "start")
        
        out_Animal_ParentOf @recurse(depth: 4) {
            name @output(out_name: "first_recursion")
            
            in_Animal_ParentOf @recurse(depth: 4) {
                name @output(out_name: "second_recursion")
            }
        }
    }
}"""
    query_arguments = {}
    ir_and_metadata = graphql_to_ir(schema, depth_2)
    expected_result = list(interpret_ir(InMemoryAdapter(), ir_and_metadata, query_arguments))
    
    for query in (depth_3, depth_4):
        ir_and_metadata = graphql_to_ir(schema, query)
        actual_result = list(interpret_ir(InMemoryAdapter(), ir_and_metadata, query_arguments))
        
        if expected_result != actual_result:
            raise AssertionError(f"Result mimatch on query: {query}")
            
    print("Success!")
            
recursion_sanity_check()

Success!


In [5]:
query1 = '''
{
    Animal {
        name @output(out_name: "animal_name")
        uuid @output(out_name: "animal_uuid")
    }
}
'''
query_arguments1 = {}

In [6]:
query2 = '''
{
    Animal {
        name @output(out_name: "parent_name")

        out_Animal_ParentOf {
            name @output(out_name: "child_name")
        }
    }
}
'''
query_arguments2 = {}

In [7]:
query3 = '''{
    Animal {
        name @output(out_name: "parent_name")
        
        out_Animal_ParentOf @optional {
            name @filter(op_name: "in_collection", value: ["$child_names"])
                 @output(out_name: "child_name")
        }
    }
}'''
query_arguments3 = {
    "child_names": ['Domino', 'Dipstick', 'Oddball'],
}

In [8]:
query4 = '''{
    Animal {
        name @output(out_name: "ancestor_or_self_name")
        
        out_Animal_ParentOf @recurse(depth: 4) {
            name @filter(op_name: "in_collection", value: ["$names"])
                 @output(out_name: "descendant_or_self_name")
        }
    }
}'''
query_arguments4 = {
    "names": ['Domino', 'Dipstick', 'Oddball'],
}

In [9]:
query5 = '''{
    Animal {
        name @output(out_name: "ancestor_or_self_name")
        
        out_Animal_ParentOf @recurse(depth: 1) {
            name @filter(op_name: "in_collection", value: ["$names"])
                 @output(out_name: "descendant_or_self_name")
        }
    }
}'''
query_arguments5 = {
    "names": ['Domino', 'Dipstick', 'Oddball'],
}

In [10]:
query6 = '''
{
    Animal {
        name @output(out_name: "parent_name")

        out_Animal_ParentOf {
            name @filter(op_name: "in_collection", value: ["$child_names"])
                 @output(out_name: "child_name")
        }
    }
}
'''
query_arguments6 = {
    "child_names": ['Domino', 'Dipstick', 'Oddball'],
}

In [11]:
query7 = '''{
    Animal {
        name @output(out_name: "ancestor_or_self_name")
        
        out_Animal_ParentOf @recurse(depth: 4) {
            name @output(out_name: "descendant_or_self_name")
        }
    }
}'''
query_arguments7 = {}

In [12]:
query8 = '''{
    Animal {
        name @output(out_name: "start") @filter(op_name: "=", value: ["$start"])
        
        in_Animal_ParentOf @recurse(depth: 4) {
            name @output(out_name: "first_recursion")
        }
    }
}'''
query_arguments8ths  = {
    "start": "Dipstick"
}

In [13]:
query9 = '''{
    Animal {
        name @output(out_name: "start") @filter(op_name: "=", value: ["$start"])
        
        out_Animal_ParentOf @recurse(depth: 3) {
            name @output(out_name: "first_recursion")
            
            in_Animal_ParentOf @recurse(depth: 3) {
                name @output(out_name: "second_recursion")
            }
        }
    }
}'''
query_arguments9 = {
    "start": "Perdy"
}

In [14]:
query10 = '''{
    Animal {
        name @output(out_name: "start")
        
        out_Animal_ParentOf @recurse(depth: 3) {
            name @output(out_name: "first_recursion")
            
            in_Animal_ParentOf @recurse(depth: 3) {
                name @output(out_name: "second_recursion")
            }
        }
    }
}'''
query_arguments10 = {}

In [15]:
query11 = '''
{
    Animal {
        name @output(out_name: "grandparent_name")

        out_Animal_ParentOf {
            name @output(out_name: "parent_name")
            
            out_Animal_ParentOf {
                name @output(out_name: "child_name")
            }
        }
    }
}
'''
query_arguments11 = {}

In [16]:
query12 = '''
{
    Animal {
        name @output(out_name: "animal_name") @filter(op_name: "=", value: ["$name"])
        uuid @output(out_name: "animal_uuid")
        
        out_Animal_ParentOf {
            name @output(out_name: "child_name")
        }
        in_Animal_ParentOf {
            name @output(out_name: "parent_name")
        }
    }
}
'''
query_arguments12 = {
    "name": "Dipstick",
}

In [17]:
ir_and_metadata = graphql_to_ir(schema, query1)
args = query_arguments1
# pprint(ir_and_metadata.ir_blocks)

adapter = InterpreterAdapterTap(InMemoryAdapter())
result_gen = interpret_ir(adapter, ir_and_metadata, args)

In [18]:
next(result_gen)

{'animal_name': 'Scooby Doo', 'animal_uuid': '1001'}

In [19]:
trace = adapter.recorder.get_trace()
list(enumerate(trace.operations))

[(0,
  AdapterOperation(kind='call', name='project_property', uid=0, parent_uid=-1, data=(('__input_iterable', 'Animal', 'uuid'), {}))),
 (1,
  AdapterOperation(kind='call', name='project_property', uid=1, parent_uid=-1, data=(('__input_iterable', 'Animal', 'name'), {}))),
 (2,
  AdapterOperation(kind='call', name='get_tokens_of_type', uid=2, parent_uid=-1, data=(('Animal',), {'runtime_arg_hints': {}, 'used_property_hints': frozenset({'name', 'uuid'}), 'filter_hints': [], 'neighbor_hints': []}))),
 (3,
  AdapterOperation(kind='yield', name='get_tokens_of_type', uid=3, parent_uid=2, data={'name': 'Scooby Doo', 'uuid': '1001'})),
 (4,
  AdapterOperation(kind='yield', name='__input_iterable', uid=4, parent_uid=1, data=DataContext(current={'name': 'Scooby Doo', 'uuid': '1001'}, locations={Location(('Animal',), None, 1): {'name': 'Scooby Doo', 'uuid': '1001'}}, stack=ImmutableStack(value=DataContext(current={'name': 'Scooby Doo', 'uuid': '1001'}, locations={Location(('Animal',), None, 1): {

In [20]:
list(result_gen)
trace = adapter.recorder.get_trace()

In [21]:
list(enumerate(trace.operations))

[(0,
  AdapterOperation(kind='call', name='project_property', uid=0, parent_uid=-1, data=(('__input_iterable', 'Animal', 'uuid'), {}))),
 (1,
  AdapterOperation(kind='call', name='project_property', uid=1, parent_uid=-1, data=(('__input_iterable', 'Animal', 'name'), {}))),
 (2,
  AdapterOperation(kind='call', name='get_tokens_of_type', uid=2, parent_uid=-1, data=(('Animal',), {'runtime_arg_hints': {}, 'used_property_hints': frozenset({'name', 'uuid'}), 'filter_hints': [], 'neighbor_hints': []}))),
 (3,
  AdapterOperation(kind='yield', name='get_tokens_of_type', uid=3, parent_uid=2, data={'name': 'Scooby Doo', 'uuid': '1001'})),
 (4,
  AdapterOperation(kind='yield', name='__input_iterable', uid=4, parent_uid=1, data=DataContext(current={'name': 'Scooby Doo', 'uuid': '1001'}, locations={Location(('Animal',), None, 1): {'name': 'Scooby Doo', 'uuid': '1001'}}, stack=ImmutableStack(value=DataContext(current={'name': 'Scooby Doo', 'uuid': '1001'}, locations={Location(('Animal',), None, 1): {

In [22]:
replay_adapter = TraceReplayAdapter(trace)

In [23]:
result_gen = interpret_ir(replay_adapter, ir_and_metadata, args)

In [24]:
list(result_gen)

[{'animal_name': 'Scooby Doo', 'animal_uuid': '1001'},
 {'animal_name': 'Hedwig', 'animal_uuid': '1002'},
 {'animal_name': 'Beethoven', 'animal_uuid': '1003'},
 {'animal_name': 'Pongo', 'animal_uuid': '1004'},
 {'animal_name': 'Perdy', 'animal_uuid': '1005'},
 {'animal_name': 'Dipstick', 'animal_uuid': '1006'},
 {'animal_name': 'Dottie', 'animal_uuid': '1007'},
 {'animal_name': 'Domino', 'animal_uuid': '1008'},
 {'animal_name': 'Little Dipper', 'animal_uuid': '1009'},
 {'animal_name': 'Oddball', 'animal_uuid': '1010'}]